# Домашнее задание к лекции "Основы веб-скрапинга и работы с API"

## Задание 1. 

### Обязательная часть

Будем парсить страницу со свежеми новостям на [habr.com/ru/all/](https://habr.com/ru/all/).

Вам необходимо собирать только те статьи, в которых встречается хотя бы одно требуемое ключевое слово. Эти слова определяем в начале кода в переменной, например:

`KEYWORDS = ['python', 'парсинг']`

 Поиск вести по всей доступной preview-информации (это информация, доступная непосредственно с текущей страницы). 
 
В итоге должен формироваться датафрейм со столбцами: <дата> - <заголовок> - <ссылка>.

### Дополнительная часть (необязательная)

Улучшить скрипт так, чтобы он анализировал не только preview-информацию статьи, но и весь текст статьи целиком.

Для этого потребуется получать страницы статей и искать по тексту внутри этой страницы.

Итоговый датафрейм формировать со столбцами: <дата> - <заголовок> - <ссылка> - <текст статьи>

In [1]:
import requests
import time
import pandas as pd
from datetime import datetime, timedelta
from bs4 import BeautifulSoup

In [2]:
KEYWORDS = ['python', 'парсинг']

In [3]:
req = requests.get('https://habr.com/ru/all/')
soup = BeautifulSoup(req.text, 'html.parser')

In [4]:
posts = soup.find_all('article', class_='post')
# posts

In [5]:
links = []

for post in posts:
    url = post.find('a', class_='post__title_link')
    a = url.attrs.get('href')
    links.append(a)

list_ = []

for link in links:
    req = requests.get(link)
    soup = BeautifulSoup(req.text, 'html.parser')
    news = soup.find_all('div', class_='post__text')
    
    for el in news:
        el_lower = el.text.lower()  
        # ищем вхождение ключевого слова в посте
        for words in KEYWORDS:
            if words in el_lower:
                data = soup.find('span', class_='post__time').attrs.get('data-time_published')
                post_title = soup.find('span', class_='post__title-text').text
                href = link
                text = soup.find('div', class_='post__text').text
                row = {'data': data, 'title': post_title, 'href': href, 'text': text} 
                list_.append(row) 
                break   
                
df = pd.DataFrame(list_)
df['data'] = df['data'].astype('datetime64')
df

,data,title,href,text
0,2020-08-03 11:06:00,Canary Deployment в Kubernetes #3: Istio,https://habr.com/ru/company/nixys/blog/513578/,Использование Istio+Kiali для запуска и визуал...
1,2020-08-03 09:17:00,Быстрый старт и низкий потолок. Что ждет молод...,https://habr.com/ru/company/skillfactory/blog/...,По исследованиям HeadHunter и Mail.ru спрос на...
2,2020-08-03 09:16:00,Так ли безопасен ваш iPhone? Обзор уязвимостей...,https://habr.com/ru/company/ruvds/blog/513562/,\n\r\nПользовательские данные не являются разм...
3,2020-08-03 09:01:00,Как я выкинул webpack и написал babel-plugin д...,https://habr.com/ru/post/513564/,Предыстория\r\nКак-то субботним вечером я сиде...
4,2020-08-03 08:30:00,Как перезапустить закон Мура программными мето...,https://habr.com/ru/company/vdsina/blog/513436/,Профессор Никлаус Вирт был прав. Создатель язы...


## Задание 2.

### Обязательная часть

Написать скрипт, который будет проверять список e-mail адресов на утечку при помощи сервиса [Avast Hack Ckeck](https://www.avast.com/hackcheck/).
Список email-ов задаем переменной в начале кода:  
`EMAIL = [xxx@x.ru, yyy@y.com]`

В итоге должен формироваться датафрейм со столбцами: <почта> - <дата утечки> - <источник утечки> - <описание утечки>.

In [6]:
email_list = ['eterin@agriculture.ru',
         'lawpro-707@list.ru',
         'solowey011@yandex.ru',
         'ahds@aol.com',
         'akas@aol.com',
         'workpost@fxt.ru'
]

In [7]:
def check_email_list(email_list):
    list_of_leaks = []

    for email in email_list:
        url = 'https://digibody.avast.com/v1/web/leaks'
        params = {'email': email}
        req = requests.post(url, json=params)
        time.sleep(0.3)
        leaks = req.json()['value']

        for leak in leaks:
            email_2 = email
            date = leak['leak_info']['date']
            source = leak['leak_info']['title']
            description = leak['leak_info']['description'] 
            row = {'email': email_2, 'date': date, 'source': source, 'description': description} 
            list_of_leaks.append(row)
    df = pd.DataFrame(list_of_leaks)   
    return df      

In [8]:
table = check_email_list(email_list)

In [9]:
table.head()

,email,date,source,description
0,lawpro-707@list.ru,1565222400000,Borealis.su,"At an unconfirmed date, Russian Minecraft serv..."
1,lawpro-707@list.ru,1588204800000,Payad,"At an unconfirmed date, the advertising revenu..."
2,solowey011@yandex.ru,1549411200000,Collection #5 Combo List,"On January 7, 2019, an online user named Sanix..."
3,solowey011@yandex.ru,1548720000000,Collection #2 Combo List,"On January 7, 2019, an online user named Sanix..."
4,solowey011@yandex.ru,1549411200000,2019 Antipublic Combo List,"On January 7, 2019, an online user named Sanix..."


In [10]:
def data_conversion(data):
    d = datetime.fromtimestamp(int(str(data)[0:10])).strftime('%Y-%m-%d')
    return d

In [11]:
table['date'] = table['date'].apply(data_conversion)

In [12]:
table.head()

,email,date,source,description
0,lawpro-707@list.ru,2019-08-08,Borealis.su,"At an unconfirmed date, Russian Minecraft serv..."
1,lawpro-707@list.ru,2020-04-30,Payad,"At an unconfirmed date, the advertising revenu..."
2,solowey011@yandex.ru,2019-02-06,Collection #5 Combo List,"On January 7, 2019, an online user named Sanix..."
3,solowey011@yandex.ru,2019-01-29,Collection #2 Combo List,"On January 7, 2019, an online user named Sanix..."
4,solowey011@yandex.ru,2019-02-06,2019 Antipublic Combo List,"On January 7, 2019, an online user named Sanix..."


### Дополнительная часть (необязательная)

Написать скрипт, который будет получать 50 последних постов указанной группы во Вконтакте.  
Документация к API VK: https://vk.com/dev/methods
, вам поможет метод [wall.get](https://vk.com/dev/wall.get)```
GROUP = 'netology'
TOKEN = УДАЛЯЙТЕ В ВЕРСИИ ДЛЯ ПРОВЕРКИ, НА GITHUB НЕ ВЫКЛАДЫВАТЬ
```

В итоге должен формироваться датафрейм со столбцами: <дата поста> - <текст поста>.

In [43]:
URL = 'https://api.vk.com/method/newsfeed.search?'
GROUP = 'netology'
TOKEN = 'xxxxxxxxxxxxxxxxxxxxxxxxxxxxxxxxxxxxxxxxxxxxxxxxxxxxxxxxxxxxx'
VERSION = '5.103'

SLEEP = 0.33

In [44]:
params = {
    'access_token': TOKEN,
    'v': VERSION,
    'q': GROUP,
    'count': 50
}

In [45]:
res = requests.get(URL, params)
# res.text
# res.json()

In [46]:
df_2 = pd.DataFrame(res.json()['response']['items'])
df_2.head()

,id,date,owner_id,from_id,post_type,text,marked_as_ads,attachments,comments,likes,reposts,views,is_favorite,signer_id
0,3,1596452883,-197640082,-197640082,post,#полезности \n \nНачинаем подписываться на соо...,0.0,"[{'type': 'photo', 'photo': {'album_id': -7, '...",{'count': 0},{'count': 0},{'count': 0},{'count': 2},False,NaN
1,37534,1596445203,-66036248,-66036248,post,"Большинство email-рассылок похожи на спам, но ...",0.0,"[{'type': 'photo', 'photo': {'album_id': -7, '...",{'count': 0},{'count': 38},{'count': 4},{'count': 2990},False,NaN
2,1182,1596443343,-44983992,-44983992,post,Новое и интересное! Лови подборку лекций и веб...,0.0,"[{'type': 'photo', 'photo': {'album_id': -7, '...",{'count': 0},{'count': 0},{'count': 0},{'count': 36},False,NaN
3,2720,1596430335,-175581292,-175581292,post,💻📱 Дистанционные образовательные площадки с ог...,0.0,"[{'type': 'photo', 'photo': {'album_id': -7, '...",{'count': 0},{'count': 3},{'count': 0},{'count': 48},False,NaN
4,243,1596294720,-186521205,-186521205,post,Как начинающему специалисту получить первую ра...,0.0,"[{'type': 'photo', 'photo': {'album_id': -7, '...",{'count': 0},{'count': 0},{'count': 0},{'count': 5},False,NaN


In [47]:
df_2['date'] = df_2['date'].apply(data_conversion)
df_2.head()

,id,date,owner_id,from_id,post_type,text,marked_as_ads,attachments,comments,likes,reposts,views,is_favorite,signer_id
0,3,2020-08-03,-197640082,-197640082,post,#полезности \n \nНачинаем подписываться на соо...,0.0,"[{'type': 'photo', 'photo': {'album_id': -7, '...",{'count': 0},{'count': 0},{'count': 0},{'count': 2},False,NaN
1,37534,2020-08-03,-66036248,-66036248,post,"Большинство email-рассылок похожи на спам, но ...",0.0,"[{'type': 'photo', 'photo': {'album_id': -7, '...",{'count': 0},{'count': 38},{'count': 4},{'count': 2990},False,NaN
2,1182,2020-08-03,-44983992,-44983992,post,Новое и интересное! Лови подборку лекций и веб...,0.0,"[{'type': 'photo', 'photo': {'album_id': -7, '...",{'count': 0},{'count': 0},{'count': 0},{'count': 36},False,NaN
3,2720,2020-08-03,-175581292,-175581292,post,💻📱 Дистанционные образовательные площадки с ог...,0.0,"[{'type': 'photo', 'photo': {'album_id': -7, '...",{'count': 0},{'count': 3},{'count': 0},{'count': 48},False,NaN
4,243,2020-08-02,-186521205,-186521205,post,Как начинающему специалисту получить первую ра...,0.0,"[{'type': 'photo', 'photo': {'album_id': -7, '...",{'count': 0},{'count': 0},{'count': 0},{'count': 5},False,NaN


In [41]:
news_netology = df_2[['date', 'text']]
news_netology.head()

,date,text
0,2020-08-03,#полезности \n \nНачинаем подписываться на соо...
1,2020-08-03,"Большинство email-рассылок похожи на спам, но ..."
2,2020-08-03,Новое и интересное! Лови подборку лекций и веб...
3,2020-08-03,💻📱 Дистанционные образовательные площадки с ог...
4,2020-08-02,Как начинающему специалисту получить первую ра...
